# gu_toolkit: Efficient Guide, Compendium, and Applications

This notebook is an opinionated, practical guide for everyday toolkit use. It is organized for **fast onboarding** and **copy/paste utility**.

## How to use this notebook
1. Run the setup cell.
2. Jump to the section you need (quickstart, recipes, applications).
3. Copy a pattern and adapt it.

---

## Contents
- Quickstart
- Frequently used APIs (compendium)
- Parameter and numeric-expression patterns
- Introspection and debugging helpers
- Cool applications (interactive demos)


In [ ]:
# Find where the package root is
import conftest

In [ ]:
# Setup: Import toolkit
from gu_toolkit import *

# 1) Quickstart

## Minimal interactive figure

In [ ]:
fig = Figure(x_range=(-8, 8))
display(fig)

with fig:
    set_title("Quickstart: Interactive sine")
    plot(a*sin(b*x), x, id="wave", label="a*sin(bx)")

## Figure with some configuration

In [ ]:
fig = Figure(x_range=(-3*pi, 3* pi))
display(fig)

with fig:
    set_title("Quickstart: Interacting sine waves")
    plot(sin(b[1]*x), x, id="wave1", label="sin(b_1x)", color="blue", opacity=0.075, thickness=5)
    plot(sin(b[2]*x), x, id="wave2", label="sin(b_2x)", color="green", alpha=0.075, width=5)
    # alpha is an alias for opacity, width is an alias for thickness
    plot(sin(b[1]*x)*sin(b[2]*x), x, id="product_wave", label="sin(b_1x) sin(b_2x)", color="red", opacity=1)
    # color can be specified as a name or a hex value
    parameter(b[1], value=1, min=1, max=20, step=1)
    parameter(b[2], value=2, min=1, max=20, step=1)


## Saving Figure code

Make sure to run the code to create the figure. Play around with it and decide on specific values of the parameters. Later `figure.to_code()` can be used to print out the code that will generate the same figure

In [ ]:
print(fig.code)
# The generated snippet captures the current figure, parameters, and traces so it can be reused in another notebook.
# More options for code generation can be specified by using fig.to_code()

#BUG: by default, do 
# from gu_toolkit import *
# that includes sp as a top level import so you should just use sin(x) instead of sp.sin(x)
# but include an option for minimal inclusions that give what . 

#BUG: gu_toolkit provides SymbolFamily and FunctionFamily that allow to write b[1] instead of b_1.
# By default use that instread of sympy symbols (but provide a code generation configuration option to keep your version)

#BUG: do no include default values like y_range or sampling_points if they were not provided. 

#BUG: if the ranges were specified using sympy values, develop a system that keeps track of that instead of writing numerical values all the time. 
# Here, we would need to have 3*pi instead. 
# To be fair, this is a big change.

## Plotting a purely numeric function or a python callable

The callable-first API supports plain Python functions as long as they can evaluate over NumPy arrays.


In [ ]:
import numpy as np

x = SymbolFamily("x")
fig_callable = Figure(x_range=(-6, 6), y_range=(-1.5, 1.5), sampling_points=500)
display(fig_callable)

with fig_callable:
    set_title("Numeric callable: damped oscillation")

    def damped_wave(x):
        return np.exp(-0.15 * x**2) * np.cos(3 * x)

    # For single-variable callables, pass the plotting symbol as the second argument.
    plot(damped_wave, x, id="damped", color="teal", label="exp(-0.15x^2) cos(3x)")
#BUG: see stacktrace below

BUG:
```python
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[12], line 14
     11     return np.exp(-0.15 * x**2) * np.cos(3 * x)
     13 # For single-variable callables, pass the plotting symbol as the second argument.
---> 14 plot(damped_wave, x, id="damped", color="teal", label="exp(-0.15x^2) cos(3x)")

File ~\Documents\notebooks\gu_toolkit\Figure.py:1875, in plot(func, var, parameters, id, label, visible, x_domain, sampling_points, color, thickness, width, dash, opacity, alpha, line, trace, view, vars)
   1873     fig = Figure()
   1874     display(fig)
-> 1875 return fig.plot(
   1876     func,
   1877     var,
   1878     parameters=parameters,
   1879     id=id,
   1880     label=label,
   1881     visible=visible,
   1882     x_domain=x_domain,
   1883     sampling_points=sampling_points,
   1884     color=color,
   1885     thickness=thickness,
   1886     width=width,
   1887     dash=dash,
   1888     line=line,
   1889     opacity=opacity,
   1890     alpha=alpha,
   1891     trace=trace,
   1892     view=view,
   1893     vars=vars,
   1894 )

File ~\Documents\notebooks\gu_toolkit\Figure.py:1122, in Figure.plot(self, func, var, parameters, id, label, visible, x_domain, sampling_points, color, thickness, width, dash, line, opacity, alpha, trace, view, vars)
   1120 if normalized_numeric_fn is not None:
   1121     plot.set_numeric_function(normalized_var, normalized_numeric_fn, parameters=parameters)
-> 1122     plot.render()
   1124 return plot

File ~\Documents\notebooks\gu_toolkit\figure_plot.py:779, in Plot.render(self, view_id)
    777 # 3. Compute
    778 x_values = np.linspace(x_min, x_max, num=int(num))
--> 779 y_values = np.asarray(self.numeric_expression(x_values))
    780 self._x_data = x_values.copy()
    781 self._y_data = y_values.copy()

File ~\Documents\notebooks\gu_toolkit\numpify.py:318, in NumericFunction.__call__(self, *positional_args, **keyed_args)
    314     raise ValueError(
    315         f"Dynamic var {sym!r} ('{var_name}') requires parameter_context at call time"
    316     )
    317 if sym not in self._parameter_context:
--> 318     raise KeyError(
    319         f"parameter_context is missing symbol {sym!r} ('{var_name}')"
    320     )
    321 full_values.append(self._parameter_context[sym])
    322 continue

KeyError: "parameter_context is missing symbol x ('x')"
QueuedDebouncer callback failed
Traceback (most recent call last):
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\debouncing.py", line 126, in _on_tick
    self._callback(*call.args, **call.kwargs)
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 478, in <lambda>
    lambda *args: self._run_relayout(view_id=view_id, *args),
                  ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 1470, in _run_relayout
    self.render(reason="relayout")
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 1191, in render
    plot.render(view_id=self.active_view_id)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\figure_plot.py", line 779, in render
    y_values = np.asarray(self.numeric_expression(x_values))
                          ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\numpify.py", line 318, in __call__
    raise KeyError(
        f"parameter_context is missing symbol {sym!r} ('{var_name}')"
    )
KeyError: "parameter_context is missing symbol x ('x')"
```

In [ ]:
fig_callable_params = Figure(x_range=(-8, 8), y_range=(-2.5, 2.5), sampling_points=600)
display(fig_callable_params)

with fig_callable_params:
    set_title("Callable with explicit vars + sliders")
    plot(lambda x, A, k: A * sin(k * x), x, vars=(x, A, k), id="param-wave", color="orange")
    # For multi-argument callables, preferably declare vars=(...) so slider-backed parameters are mapped explicitly.
    plot(lambda x, A, k: A+ k*x , x, vars={'x':x,'A':A,'k':k[2]}, id="param-line", color="blue")
    parameter(A, value=1.2, min=0.0, max=2.5, step=0.1)
    parameter(k, value=1.0, min=0.5, max=4.0, step=0.1)

#BUG: callable dictionary api does not work. See stack trace below
#ISSUE TO PONDER: There is abuse happening in vars: we pass symbols but they get interpreted in strings. Raise issue to make sure this procedure is not fragile. 

BUG STACK TRACE:
```python
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File C:\opt\Python\current\App\Lib\site-packages\sympy\core\cache.py:72, in __cacheit.<locals>.func_wrapper.<locals>.wrapper(*args, **kwargs)
     71 try:
---> 72     retval = cfunc(*args, **kwargs)
     73 except TypeError as e:

TypeError: unhashable type: 'numpy.ndarray'

During handling of the above exception, another exception occurred:

TypeError                                 Traceback (most recent call last)
File C:\opt\Python\current\App\Lib\site-packages\sympy\core\cache.py:72, in __cacheit.<locals>.func_wrapper.<locals>.wrapper(*args, **kwargs)
     71 try:
---> 72     retval = cfunc(*args, **kwargs)
     73 except TypeError as e:

TypeError: unhashable type: 'numpy.ndarray'

During handling of the above exception, another exception occurred:

AttributeError                            Traceback (most recent call last)
Cell In[12], line 6
      4 with fig_callable_params:
      5     set_title("Callable with explicit vars + sliders")
----> 6     plot(lambda x, A, k: A * sin(k * x), x, vars=(x, A, k), id="param-wave", color="orange")
      7     # For multi-argument callables, preferably declare vars=(...) so slider-backed parameters are mapped explicitly.
      8     plot(lambda x, A, k: A+ k*x , x, vars={'x':x,'A':A,'k':k[2]}, id="param-line", color="blue")

File ~\Documents\notebooks\gu_toolkit\Figure.py:1875, in plot(func, var, parameters, id, label, visible, x_domain, sampling_points, color, thickness, width, dash, opacity, alpha, line, trace, view, vars)
   1873     fig = Figure()
   1874     display(fig)
-> 1875 return fig.plot(
   1876     func,
   1877     var,
   1878     parameters=parameters,
   1879     id=id,
   1880     label=label,
   1881     visible=visible,
   1882     x_domain=x_domain,
   1883     sampling_points=sampling_points,
   1884     color=color,
   1885     thickness=thickness,
   1886     width=width,
   1887     dash=dash,
   1888     line=line,
   1889     opacity=opacity,
   1890     alpha=alpha,
   1891     trace=trace,
   1892     view=view,
   1893     vars=vars,
   1894 )

File ~\Documents\notebooks\gu_toolkit\Figure.py:1122, in Figure.plot(self, func, var, parameters, id, label, visible, x_domain, sampling_points, color, thickness, width, dash, line, opacity, alpha, trace, view, vars)
   1120 if normalized_numeric_fn is not None:
   1121     plot.set_numeric_function(normalized_var, normalized_numeric_fn, parameters=parameters)
-> 1122     plot.render()
   1124 return plot

File ~\Documents\notebooks\gu_toolkit\figure_plot.py:779, in Plot.render(self, view_id)
    777 # 3. Compute
    778 x_values = np.linspace(x_min, x_max, num=int(num))
--> 779 y_values = np.asarray(self.numeric_expression(x_values))
    780 self._x_data = x_values.copy()
    781 self._y_data = y_values.copy()

File ~\Documents\notebooks\gu_toolkit\numpify.py:348, in NumericFunction.__call__(self, *positional_args, **keyed_args)
    345 if extra:
    346     raise TypeError(f"Unknown keyed argument(s): {', '.join(sorted(extra))}")
--> 348 return self._fn(*full_values)

Cell In[12], line 6, in <lambda>(x, A, k)
      4 with fig_callable_params:
      5     set_title("Callable with explicit vars + sliders")
----> 6     plot(lambda x, A, k: A * sin(k * x), x, vars=(x, A, k), id="param-wave", color="orange")
      7     # For multi-argument callables, preferably declare vars=(...) so slider-backed parameters are mapped explicitly.
      8     plot(lambda x, A, k: A+ k*x , x, vars={'x':x,'A':A,'k':k[2]}, id="param-line", color="blue")

File C:\opt\Python\current\App\Lib\site-packages\sympy\core\cache.py:76, in __cacheit.<locals>.func_wrapper.<locals>.wrapper(*args, **kwargs)
     74     if not e.args or not e.args[0].startswith('unhashable type:'):
     75         raise
---> 76     retval = func(*args, **kwargs)
     77 return retval

File C:\opt\Python\current\App\Lib\site-packages\sympy\core\function.py:823, in DefinedFunction.__new__(cls, *args, **options)
    821 @cacheit
    822 def __new__(cls, *args, **options) -> Expr:  # type: ignore
--> 823     return cls._new_(*args, **options)

File C:\opt\Python\current\App\Lib\site-packages\sympy\core\function.py:472, in Function._new_(cls, *args, **options)
    464     raise TypeError(temp % {
    465         'name': cls,
    466         'qual': 'exactly' if len(cls.nargs) == 1 else 'at least',
    467         'args': min(cls.nargs),
    468         'plural': 's'*(min(cls.nargs) != 1),
    469         'given': n})
    471 evaluate = options.get('evaluate', global_parameters.evaluate)
--> 472 result = super().__new__(cls, *args, **options)
    473 if evaluate and isinstance(result, cls) and result.args:
    474     _should_evalf = [cls._should_evalf(a) for a in result.args]

File C:\opt\Python\current\App\Lib\site-packages\sympy\core\cache.py:76, in __cacheit.<locals>.func_wrapper.<locals>.wrapper(*args, **kwargs)
     74     if not e.args or not e.args[0].startswith('unhashable type:'):
     75         raise
---> 76     retval = func(*args, **kwargs)
     77 return retval

File C:\opt\Python\current\App\Lib\site-packages\sympy\core\function.py:309, in Application.__new__(cls, *args, **options)
    306     raise ValueError("Unknown options: %s" % options)
    308 if evaluate:
--> 309     evaluated = cls.eval(*args)
    310     if evaluated is not None:
    311         return evaluated

File C:\opt\Python\current\App\Lib\site-packages\sympy\functions\elementary\trigonometric.py:343, in sin.eval(cls, arg)
    340 if arg.could_extract_minus_sign():
    341     return -cls(-arg)
--> 343 i_coeff = _imaginary_unit_as_coefficient(arg)
    344 if i_coeff is not None:
    345     from sympy.functions.elementary.hyperbolic import sinh

File C:\opt\Python\current\App\Lib\site-packages\sympy\functions\elementary\trigonometric.py:38, in _imaginary_unit_as_coefficient(arg)
     36     return None
     37 else:
---> 38     return arg.as_coefficient(S.ImaginaryUnit)

AttributeError: 'ImmutableDenseNDimArray' object has no attribute 'as_coefficient'
QueuedDebouncer callback failed
Traceback (most recent call last):
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\core\cache.py", line 72, in wrapper
    retval = cfunc(*args, **kwargs)
TypeError: unhashable type: 'numpy.ndarray'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\core\cache.py", line 72, in wrapper
    retval = cfunc(*args, **kwargs)
TypeError: unhashable type: 'numpy.ndarray'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\debouncing.py", line 126, in _on_tick
    self._callback(*call.args, **call.kwargs)
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 478, in <lambda>
    lambda *args: self._run_relayout(view_id=view_id, *args),
                  ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 1470, in _run_relayout
    self.render(reason="relayout")
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 1191, in render
    plot.render(view_id=self.active_view_id)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\figure_plot.py", line 779, in render
    y_values = np.asarray(self.numeric_expression(x_values))
                          ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^
  File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\numpify.py", line 348, in __call__
    return self._fn(*full_values)
           ~~~~~~~~^^^^^^^^^^^^^^
  File "C:\Users\guraltsev\AppData\Local\Temp\ipykernel_18020\903295117.py", line 6, in <lambda>
    plot(lambda x, A, k: A * sin(k * x), x, vars=(x, A, k), id="param-wave", color="orange")
                             ~~~^^^^^^^
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\core\cache.py", line 76, in wrapper
    retval = func(*args, **kwargs)
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\core\function.py", line 823, in __new__
    return cls._new_(*args, **options)
           ~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\core\function.py", line 472, in _new_
    result = super().__new__(cls, *args, **options)
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\core\cache.py", line 76, in wrapper
    retval = func(*args, **kwargs)
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\core\function.py", line 309, in __new__
    evaluated = cls.eval(*args)
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\functions\elementary\trigonometric.py", line 343, in eval
    i_coeff = _imaginary_unit_as_coefficient(arg)
  File "C:\opt\Python\current\App\Lib\site-packages\sympy\functions\elementary\trigonometric.py", line 38, in _imaginary_unit_as_coefficient
    return arg.as_coefficient(S.ImaginaryUnit)
           ^^^^^^^^^^^^^^^^^^
AttributeError: 'ImmutableDenseNDimArray' object has no attribute 'as_coefficient'
```

## Quickstart: adding non-trivial info cards

Info cards can mix static HTML with dynamic callback segments that re-evaluate on render and slider changes.
Use this pattern to show diagnostics, derived metrics, and runtime context next to your plot.


Info cards let you attach rich, always-visible context to the figure panel. Use them for interpretation notes, formulas, and operating instructions so users can understand controls without leaving the notebook.

Practical guidance:
- Keep one card for *how to interact* (which slider controls what).
- Keep one card for *what to look for* (expected shape changes, limits, or invariants).
- Prefer short HTML snippets (`<b>`, `<code>`, `<br>`) so cards remain compact in narrow sidebars.


In [ ]:
fig_info = Figure(x_range=(-2*sp.pi, 2*sp.pi), y_range=(-3, 3), sampling_points=600)
display(fig_info)

with fig_info:
    set_title("Quickstart: plot + diagnostics info cards")
    plot(a*sin(b*x), x, id="main-wave", label="a*sin(bx)", color="royalblue")
    parameter(a, value=1.0, min=0.2, max=2.0, step=0.1)
    parameter(b, value=1.0, min=0.5, max=4.0, step=0.1)

    info("""
    <b>Tips</b><br>
    • Move <code>a</code> to scale amplitude.<br>
    • Move <code>b</code> to change frequency.<br>
    """, id="usage-tips")

    def diagnostics_card(fig, ctx):
        p = fig.params.snapshot()
        amp = p[a]
        freq = p['b']
        period = (2*np.pi/freq) if freq else float("inf")
        est_energy = 0.5 * amp**2
        return (
            f"<b>Wave diagnostics</b><br>"
            f"reason: <code>{ctx.reason}</code><br>"
            f"a={amp:.2f}, b={freq:.2f}<br>"
            f"period≈{period:.3f}, ½a²≈{est_energy:.3f}"
        )

    info(["<hr>", diagnostics_card], id="wave-diagnostics")
# params.snapshot() returns a symbol-keyed mapping; access values with the same symbols you registered.
# BUG: params.shapshot() should also support string-based access. 

## Using multiple tabs


In [ ]:
fig_tabs = Figure(x_range=(-6, 6), y_range=(-3, 3), sampling_points=400)
fig_tabs.add_view("phase", x_range=(-2*sp.pi, 2*sp.pi), y_range=(-3, 3), x_label="phase")
display(fig_tabs)

with fig_tabs:
    parameter(a, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(b, value=0.5, min=-2.0, max=2.0, step=0.1)

    plot(a*sin(x) + b, x, id="shared-wave", view=("main", "phase"), color="crimson")
    with fig_tabs.view("phase"):
        plot(sp.cos(2*x), x, id="phase-only", color="purple")

    info("<b>Shared card:</b> visible on every view", id="shared-card")
    info("<b>Phase note:</b> scoped to the phase tab", id="phase-card", view="phase")


# 2) Compendium: Frequently used operations

### Figure construction and ranges
- `Figure(...)`
- `get_x_range()`, `get_y_range()`
- `set_title(...)`

### Plot and parameter workflow
- `plot(...)`
- `parameter(symbol, ...)`
- `params[symbol]` / `params.snapshot()`
- `render()`

### Numeric expressions
- `numpify(expr, vars=...)`
- `DYNAMIC_PARAMETER` and `UNFREEZE`

### Symbolic parsing
- `parse_latex(latex_string)`


In [ ]:
#BUG: the compendium above is shit. It should be much more complete. every function should have a breif description and description of arguments, in markdown.

### B) Symbolic-to-numeric with dynamic parameters

Use this when you want to start from symbolic math and still keep interactivity.


In [ ]:
expr = parse_latex(r"A \sin(k x + \phi)")
expr.free_symbols

In [ ]:
f = numpify(expr, vars=['x', 'A', 'k', 'phi'])
f